Name: Anmol Singhal
Roll No: 11940140

References:
1. Learning Rate Schedule Taken From : https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
2. https://neptune.ai/blog/cross-validation-in-machine-learning-how-to-do-it-right

# Creating the SkLearn Pipeline for Data-Preprocessing

### Importing Libraries

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Reading Data

In [3]:
data = pd.read_csv("new-york-city-taxi-fare-prediction/train.csv", nrows = 1000000)
test_final = pd.read_csv("new-york-city-taxi-fare-prediction/test.csv")

In [4]:
# These two base classes are used to make any normal python class a sklearn class.
# 
# Every class inheriting these two classes must have to define atlease two mandatory functions
#      1. fit(self, X, y=None) -> it only learns parameters & save those as class
#                                 variables in the self object. as most of the time, it has nothing
#                                 to return, so it returns the self object only
#
#      2. transform(self, X) -> it takes input values, apply some transformation like calculation of
#                               prediction using learned parameters to make some changes in the input data.
#                               and it retuens the updated input object 
#
# there is another function as "fit_transform()", which calls "fit()" & "transform()" sequentially.
from sklearn.base import BaseEstimator, TransformerMixin

### Pipeline will Do the Following:

1. Data Cleaning
2. Adding New Features
3. Feature Selection
4. Standardization

Class for Removing Missing Values

In [5]:
class remove_missing_values(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.drop(X[X.isnull().any(1)].index, axis = 0)
        return X

Class for Removing Irrelevant Values from Data

In [6]:
class remove_irrelevant_values(BaseEstimator, TransformerMixin):
    
    def __init__(self, isFareAmount = True,
                 isPassengerCount = True,
                 isLatitudes = True,
                 isLongitudes = True):
        self.isFareAmount = isFareAmount
        self.isPassengerCount = isPassengerCount
        self.isLatitudes = isLatitudes
        self.isLongitudes = isLongitudes

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        if self.isFareAmount and 'fare_amount' in list(X.columns):
            X = X.drop(X[X['fare_amount']<=0].index, axis=0)
            
        if self.isPassengerCount:
            X = X.drop(X[X['passenger_count']<=0].index, axis=0)
            X = X.drop(X[X['passenger_count']>=7].index, axis=0)
        
        if self.isLatitudes:
            X = X.drop(X[X['pickup_latitude']<-90].index, axis=0)
            X = X.drop(X[X['pickup_latitude']>90].index, axis=0)

            X = X.drop(X[X['dropoff_latitude']<-90].index, axis=0)
            X = X.drop(X[X['dropoff_latitude']>90].index, axis=0)
        
        if self.isLongitudes:
            X = X.drop(X[X['pickup_longitude']<-180].index, axis=0)
            X = X.drop(X[X['pickup_longitude']>180].index, axis=0)

            X = X.drop(X[X['dropoff_longitude']<-180].index, axis=0)
            X = X.drop(X[X['dropoff_longitude']>180].index, axis=0)
        
        return X

Class to Remove Rows with Locations outside of a Bounding Box

In [7]:
class remove_outside_bb(BaseEstimator, TransformerMixin):
    
    def __init__(self, bb = (-74.5, -72.8, 40.5, 41.8)):
        self.bb = bb
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        self.outlier_index = X.loc[(X['pickup_longitude'] < self.bb[0]) | (X['pickup_longitude'] > self.bb[1]) | \
           (X['pickup_latitude'] < self.bb[2]) | (X['pickup_latitude'] > self.bb[3]) | \
           (X['dropoff_longitude'] < self.bb[0]) | (X['dropoff_longitude'] > self.bb[1]) | \
           (X['dropoff_latitude'] < self.bb[2]) | (X['dropoff_latitude'] > self.bb[3])].index
        X = X.drop(self.outlier_index, axis=0)
        return X

Class to Fix Data Types

In [8]:
class fix_datetime_data_types(BaseEstimator, TransformerMixin):
    
    def __init__(self, cols = ['pickup_datetime', 'key']):
        self.cols = cols

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for col in self.cols:
            X[col] = pd.to_datetime(X[col])
        return X

Class to Add Date Time based Features

In [9]:
class add_datetime_features(BaseEstimator, TransformerMixin):
    
    def __init__(self, dt_col='pickup_datetime',
                 isHour = False,
                 isDayOfWeek = False,
                 isDayOfMonth = False,
                 isWeekOfYear = False,
                 isMonth = False,
                 isYear = True):
        self.dt_col = dt_col
        self.isHour = isHour
        self.isDayOfWeek = isDayOfWeek
        self.isDayOfMonth = isDayOfMonth
        self.isWeekOfYear = isWeekOfYear
        self.isMonth = isMonth
        self.isYear = isYear

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if self.isHour:
            X["hour"] = X[self.dt_col].dt.hour
        if self.isDayOfWeek:
            X["day_of_week"] = X[self.dt_col].dt.weekday
        if self.isDayOfMonth:
            X["day_of_month"] = X[self.dt_col].dt.day
        if self.isWeekOfYear:
            X["week"] = X[self.dt_col].dt.isocalendar().week
        if self.isMonth:
            X["month"] = X[self.dt_col].dt.month
        if self.isYear:
            X["year"] = X[self.dt_col].dt.year        
        return X

Class to remove Irrelevant Features

In [10]:
class remove_features(BaseEstimator, TransformerMixin):
    
    def __init__(self, cols = ['pickup_datetime', 'key', 'passenger_count', 'pickup_latitude', 'dropoff_latitude']):
        self.cols = cols

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for col in self.cols:
            X = X.drop(col, axis=1)
        return X

Class to Add New Features

In [11]:
class add_features(BaseEstimator, TransformerMixin):
    
    def __init__(self, isHaversine=True,
                isChebyshev=False,
                
                isDistFromJFK_pickup=True,
                isDistFromJFK_dropoff=True,
                jfk_coords = (40.639722, -73.778889),
                
                isDistFromEWR_pickup=False,
                isDistFromEWR_dropoff=False,
                ewr_coords = (40.6925, -74.168611),
                
                isDistFromLGA_pickup=False,
                isDistFromLGA_dropoff=False,
                lga_coords = (40.77725, -73.872611),
                
                isDistFromSOL_pickup=False,
                isDistFromSOL_dropoff=True,
                sol_coords = (40.689247, -74.044502),
                
                isDistFromESM_pickup=False,
                isDistFromESM_dropoff=False,
                esm_coords = (40.748440, -73.985664),
                
                isDistFromCP_pickup=True,
                isDistFromCP_dropoff=True,
                cp_coords = (40.782864, -73.965355),
                
                isDistFromTS_pickup=False,
                isDistFromTS_dropoff=False,
                ts_coords = (40.759010, -73.984474),
                
                isNearJFK_pickup=True,
                isNearJFK_dropoff=True,
                
                isNearEWR_pickup=False,
                isNearEWR_dropoff=True,

                isNearLGA_pickup=True,
                isNearLGA_dropoff=True,
                
                isNearSOL_pickup=False,
                isNearSOL_dropoff=False,
                
                isNearESM_pickup=True,
                isNearESM_dropoff=True,
                
                isNearCP_pickup=False,
                isNearCP_dropoff=False,
                
                isNearTS_pickup=True,
                isNearTS_dropoff=True,

                ):
    
        self.isHaversine = isHaversine
        self.isChebyshev = isChebyshev

        self.isDistFromJFK_pickup = isDistFromJFK_pickup
        self.isDistFromJFK_dropoff = isDistFromJFK_dropoff
        self.jfk_coords = jfk_coords

        self.isDistFromEWR_pickup = isDistFromEWR_pickup
        self.isDistFromEWR_dropoff = isDistFromEWR_dropoff
        self.ewr_coords = ewr_coords

        self.isDistFromLGA_pickup = isDistFromLGA_pickup
        self.isDistFromLGA_dropoff = isDistFromLGA_dropoff
        self.lga_coords = lga_coords
        
        self.isDistFromSOL_pickup = isDistFromSOL_pickup
        self.isDistFromSOL_dropoff = isDistFromSOL_dropoff
        self.sol_coords = sol_coords
        
        self.isDistFromESM_pickup = isDistFromESM_pickup
        self.isDistFromESM_dropoff = isDistFromESM_dropoff
        self.esm_coords = esm_coords
        
        self.isDistFromCP_pickup = isDistFromCP_pickup
        self.isDistFromCP_dropoff = isDistFromCP_dropoff
        self.cp_coords = cp_coords
        
        self.isDistFromTS_pickup = isDistFromTS_pickup
        self.isDistFromTS_dropoff = isDistFromTS_dropoff
        self.ts_coords = ts_coords

        self.isNearJFK_pickup = isNearJFK_pickup
        self.isNearJFK_dropoff = isNearJFK_dropoff

        self.isNearEWR_pickup = isNearEWR_pickup
        self.isNearEWR_dropoff = isNearEWR_dropoff

        self.isNearLGA_pickup = isNearLGA_pickup
        self.isNearLGA_dropoff = isNearLGA_dropoff
        
        self.isNearSOL_pickup = isNearSOL_pickup
        self.isNearSOL_dropoff = isNearSOL_dropoff
        
        self.isNearESM_pickup = isNearESM_pickup
        self.isNearESM_dropoff = isNearESM_dropoff
        
        self.isNearCP_pickup = isNearCP_pickup
        self.isNearCP_dropoff = isNearCP_dropoff
        
        self.isNearTS_pickup = isNearTS_pickup
        self.isNearTS_dropoff = isNearTS_dropoff

    def add_haversine_dist_from(self, dataset, airport_coordinates, lat_col, long_col, target_name):
            
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(dataset[lat_col])
        phi2 = np.radians(airport_coordinates[0])

        delta_phi = np.radians(airport_coordinates[0] - dataset[lat_col])
        delta_lambda = np.radians(airport_coordinates[1] - dataset[long_col])

        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2

        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

        #d = R*c
        d = (R * c) #in kilometers
        dataset[target_name] = d


    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        if self.isHaversine:
            
            R = 6371  #radius of earth in kilometers
            #R = 3959 #radius of earth in miles
            phi1 = np.radians(X['pickup_latitude'])
            phi2 = np.radians(X['dropoff_latitude'])

            delta_phi = np.radians(X['dropoff_latitude']-X['pickup_latitude'])
            delta_lambda = np.radians(X['dropoff_longitude']-X['pickup_longitude'])

            #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
            a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2

            #c = 2 * atan2( √a, √(1−a) )
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

            #d = R*c
            d = (R * c) #in kilometers
            X['H_Distance'] = d
        
        if self.isChebyshev:
            X['Chebyshev'] = np.maximum(np.absolute(X['dropoff_longitude'] - X['pickup_longitude']), np.absolute(X['dropoff_latitude'] - X['pickup_latitude']))
        
        if self.isDistFromJFK_pickup:
            self.add_haversine_dist_from(X, self.jfk_coords, 'pickup_latitude', 'pickup_longitude', 'jfk_Distance_Pickup')
        if self.isDistFromJFK_dropoff:
            self.add_haversine_dist_from(X, self.jfk_coords, 'dropoff_latitude', 'dropoff_longitude', 'jfk_Distance_Dropoff')
        
        if self.isDistFromEWR_pickup:
            self.add_haversine_dist_from(X, self.ewr_coords, 'pickup_latitude', 'pickup_longitude', 'ewr_Distance_Pickup')
        if self.isDistFromEWR_dropoff:
            self.add_haversine_dist_from(X, self.ewr_coords, 'dropoff_latitude', 'dropoff_longitude', 'ewr_Distance_Dropoff')
            
        if self.isDistFromLGA_pickup:
            self.add_haversine_dist_from(X, self.lga_coords, 'pickup_latitude', 'pickup_longitude', 'lga_Distance_Pickup')
        if self.isDistFromLGA_dropoff:
            self.add_haversine_dist_from(X, self.lga_coords, 'dropoff_latitude', 'dropoff_longitude', 'lga_Distance_Dropoff')
        
        if self.isDistFromSOL_pickup:
            self.add_haversine_dist_from(X, self.sol_coords, 'pickup_latitude', 'pickup_longitude', 'sol_Distance_Pickup')
        if self.isDistFromSOL_dropoff:
            self.add_haversine_dist_from(X, self.sol_coords, 'dropoff_latitude', 'dropoff_longitude', 'sol_Distance_Dropoff')
            
        if self.isDistFromESM_pickup:
            self.add_haversine_dist_from(X, self.esm_coords, 'pickup_latitude', 'pickup_longitude', 'esm_Distance_Pickup')
        if self.isDistFromESM_dropoff:
            self.add_haversine_dist_from(X, self.esm_coords, 'dropoff_latitude', 'dropoff_longitude', 'esm_Distance_Dropoff')
        
        if self.isDistFromCP_pickup:
            self.add_haversine_dist_from(X, self.cp_coords, 'pickup_latitude', 'pickup_longitude', 'cp_Distance_Pickup')
        if self.isDistFromCP_dropoff:
            self.add_haversine_dist_from(X, self.cp_coords, 'dropoff_latitude', 'dropoff_longitude', 'cp_Distance_Dropoff')
        
        if self.isDistFromTS_pickup:
            self.add_haversine_dist_from(X, self.ts_coords, 'pickup_latitude', 'pickup_longitude', 'ts_Distance_Pickup')
        if self.isDistFromTS_dropoff:
            self.add_haversine_dist_from(X, self.ts_coords, 'dropoff_latitude', 'dropoff_longitude', 'ts_Distance_Dropoff')
        
        # Considering 4 KM Distance
        DISTANCE_THRESHOLD = 4
        
        if self.isNearJFK_pickup:
            if 'jfk_Distance_Pickup' in list(X.columns):
                X['pickup_near_jfk'] = X['jfk_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from JFK to each row
                self.add_haversine_dist_from(X, self.jfk_coords, 'pickup_latitude', 'pickup_longitude', 'jfk_Distance_Pickup')
                # Now add Near JFK column
                X['pickup_near_jfk'] = X['jfk_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from JFK column
                X.drop(['jfk_Distance_Pickup'], axis=1, inplace=True)
        
        if self.isNearJFK_dropoff:
            if 'jfk_Distance_Dropoff' in list(X.columns):
                X['dropoff_near_jfk'] = X['jfk_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from JFK to each row
                self.add_haversine_dist_from(X, self.jfk_coords, 'dropoff_latitude', 'dropoff_longitude', 'jfk_Distance_Dropoff')
                # Now add Near JFK column
                X['dropoff_near_jfk'] = X['jfk_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from JFK column
                X.drop(['jfk_Distance_Dropoff'], axis=1, inplace=True)
        
        if self.isNearEWR_pickup:
            if 'ewr_Distance_Pickup' in list(X.columns):
                X['pickup_near_ewr'] = X['ewr_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from EWR to each row
                self.add_haversine_dist_from(X, self.ewr_coords, 'pickup_latitude', 'pickup_longitude', 'ewr_Distance_Pickup')
                # Now add Near EWR column
                X['pickup_near_ewr'] = X['ewr_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from EWR column
                X.drop(['ewr_Distance_Pickup'], axis=1, inplace=True)
            
        if self.isNearEWR_dropoff:
            if 'ewr_Distance_Dropoff' in list(X.columns):
                X['dropoff_near_ewr'] = X['ewr_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from EWR to each row
                self.add_haversine_dist_from(X, self.ewr_coords, 'dropoff_latitude', 'dropoff_longitude', 'ewr_Distance_Dropoff')
                # Now add Near EWR column
                X['dropoff_near_ewr'] = X['ewr_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from EWR column
                X.drop(['ewr_Distance_Dropoff'], axis=1, inplace=True)
            
        if self.isNearLGA_pickup:
            if 'lga_Distance_Pickup' in list(X.columns):
                X['pickup_near_lga'] = X['lga_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from LGA to each row
                self.add_haversine_dist_from(X, self.lga_coords, 'pickup_latitude', 'pickup_longitude', 'lga_Distance_Pickup')
                # Now add Near LGA column
                X['pickup_near_lga'] = X['lga_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from LGA column
                X.drop(['lga_Distance_Pickup'], axis=1, inplace=True)
        
        if self.isNearLGA_dropoff:
            if 'lga_Distance_Dropoff' in list(X.columns):
                X['dropoff_near_lga'] = X['lga_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from LGA to each row
                self.add_haversine_dist_from(X, self.lga_coords, 'dropoff_latitude', 'dropoff_longitude', 'lga_Distance_Dropoff')
                # Now add Near LGA column
                X['dropoff_near_lga'] = X['lga_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from LGA column
                X.drop(['lga_Distance_Dropoff'], axis=1, inplace=True)
        
        if self.isNearSOL_pickup:
            if 'sol_Distance_Pickup' in list(X.columns):
                X['pickup_near_sol'] = X['sol_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from SOL to each row
                self.add_haversine_dist_from(X, self.sol_coords, 'pickup_latitude', 'pickup_longitude', 'sol_Distance_Pickup')
                # Now add Near SOL column
                X['pickup_near_sol'] = X['sol_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from SOL column
                X.drop(['sol_Distance_Pickup'], axis=1, inplace=True)
        
        if self.isNearSOL_dropoff:
            if 'sol_Distance_Dropoff' in list(X.columns):
                X['dropoff_near_sol'] = X['sol_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from SOL to each row
                self.add_haversine_dist_from(X, self.sol_coords, 'dropoff_latitude', 'dropoff_longitude', 'sol_Distance_Dropoff')
                # Now add Near SOL column
                X['dropoff_near_sol'] = X['sol_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from SOL column
                X.drop(['sol_Distance_Dropoff'], axis=1, inplace=True)
        
        if self.isNearESM_pickup:
            if 'esm_Distance_Pickup' in list(X.columns):
                X['pickup_near_esm'] = X['esm_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from ESM to each row
                self.add_haversine_dist_from(X, self.esm_coords, 'pickup_latitude', 'pickup_longitude', 'esm_Distance_Pickup')
                # Now add Near ESM column
                X['pickup_near_esm'] = X['esm_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from ESM column
                X.drop(['esm_Distance_Pickup'], axis=1, inplace=True)
        
        if self.isNearESM_dropoff:
            if 'esm_Distance_Dropoff' in list(X.columns):
                X['dropoff_near_esm'] = X['esm_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from ESM to each row
                self.add_haversine_dist_from(X, self.esm_coords, 'dropoff_latitude', 'dropoff_longitude', 'esm_Distance_Dropoff')
                # Now add Near ESM column
                X['dropoff_near_esm'] = X['esm_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from ESM column
                X.drop(['esm_Distance_Dropoff'], axis=1, inplace=True)
        
        if self.isNearCP_pickup:
            if 'cp_Distance_Pickup' in list(X.columns):
                X['pickup_near_cp'] = X['cp_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from CP to each row
                self.add_haversine_dist_from(X, self.cp_coords, 'pickup_latitude', 'pickup_longitude', 'cp_Distance_Pickup')
                # Now add Near CP column
                X['pickup_near_cp'] = X['cp_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from CP column
                X.drop(['cp_Distance_Pickup'], axis=1, inplace=True)
        
        if self.isNearCP_dropoff:
            if 'cp_Distance_Dropoff' in list(X.columns):
                X['dropoff_near_cp'] = X['cp_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from CP to each row
                self.add_haversine_dist_from(X, self.cp_coords, 'dropoff_latitude', 'dropoff_longitude', 'cp_Distance_Dropoff')
                # Now add Near CP column
                X['dropoff_near_cp'] = X['cp_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from CP column
                X.drop(['cp_Distance_Dropoff'], axis=1, inplace=True)
        
        if self.isNearTS_pickup:
            if 'ts_Distance_Pickup' in list(X.columns):
                X['pickup_near_ts'] = X['ts_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from TS to each row
                self.add_haversine_dist_from(X, self.ts_coords, 'pickup_latitude', 'pickup_longitude', 'ts_Distance_Pickup')
                # Now add Near TS column
                X['pickup_near_ts'] = X['ts_Distance_Pickup'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from TS column
                X.drop(['ts_Distance_Pickup'], axis=1, inplace=True)
        
        if self.isNearTS_dropoff:
            if 'ts_Distance_Dropoff' in list(X.columns):
                X['dropoff_near_ts'] = X['ts_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
            else:
                # First Add Distance from TS to each row
                self.add_haversine_dist_from(X, self.ts_coords, 'dropoff_latitude', 'dropoff_longitude', 'ts_Distance_Dropoff')
                # Now add Near TS column
                X['dropoff_near_ts'] = X['ts_Distance_Dropoff'].apply(lambda x: 1 if x < DISTANCE_THRESHOLD else 0)
                # Remove Distance from TS column
                X.drop(['ts_Distance_Dropoff'], axis=1, inplace=True)
        
        return X

Class to Impute Features

In [12]:
class impute_features(BaseEstimator, TransformerMixin):
    
    def __init__(self, isHaversine=True):
        self.isHaversine = isHaversine

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):

        if self.isHaversine:
            X.loc[X['H_Distance'] == 0, 'H_Distance'] = (X['fare_amount'] - 2.50) / 1.56
        
        return X

Class to DownCast Features

In [13]:
class downcast_numeric_features(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_int = X.select_dtypes(include=['int64', 'int32', 'int16', 'int8', 'int'])
        X[X_int.columns] = X_int.apply(pd.to_numeric,downcast='unsigned')
        X_float = X.select_dtypes(include=['float64', 'float32', 'float16', 'float'])
        X[X_float.columns] = X_float.apply(pd.to_numeric,downcast='float')
        return X
    

Class to Scale the Data and Return the Final Output

In [38]:
from sklearn.preprocessing import StandardScaler

class Scaler(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        self.scaler = StandardScaler()
        # Pipeline is being fit to the dataframe with the target column in it
        if 'fare_amount' in list(X.columns):
            self.scaler.fit(X.drop(['fare_amount'], axis=1))
        else:
            # Target Column is not in the dataframe
            self.scaler.fit(X)
        return self
    
    def transform(self, X):
        print("Current Columns:", X.columns)
        # We need to perform Scaling on Dataframe but not on fare_amount
        if 'fare_amount' in list(X.columns):
            y = X['fare_amount'].to_numpy().reshape(-1,1)
            X = X.drop(['fare_amount'], axis=1)
            X = self.scaler.transform(X)
            X = np.hstack((X, y))
            return X
        else:
            return self.scaler.transform(X)
        

Class to add Ones Column to the Data

In [15]:
class add_ones(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Add a column of ones to the numpy array as the first column
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        return X

### SkLearn Pipeline for Data-Preprocessing

In [39]:
# Create a Pipeline object to apply the above class in sequence
from sklearn.pipeline import Pipeline

pre_pipeline = Pipeline([
    ('remove missing values', remove_missing_values()),
    ('remove irrelevant values', remove_irrelevant_values()),
    ('remove outside NYC', remove_outside_bb()),
    ('fix datetime data types', fix_datetime_data_types()),
    ('add datetime features', add_datetime_features()),
    ('add features', add_features()),
    ('remove extra features', remove_features()),
    ('impute features', impute_features()),
    ('downcast numeric features', downcast_numeric_features()),
    ('StandardScaler', Scaler()),
    ('Add Ones', add_ones())
])


### Splitting the Data into Training and Testing Data

In [40]:
# Split original dataset into 80% train and 20% test
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)

### Apply Pre-Processing Pipeline to the Data

Fit the Pipeline to Train Data and Transform the Data

In [41]:
train_data = pre_pipeline.fit_transform(train_data)

Current Columns: Index(['fare_amount', 'pickup_longitude', 'dropoff_longitude', 'year',
       'H_Distance', 'jfk_Distance_Pickup', 'jfk_Distance_Dropoff',
       'sol_Distance_Dropoff', 'cp_Distance_Pickup', 'cp_Distance_Dropoff',
       'pickup_near_jfk', 'dropoff_near_jfk', 'dropoff_near_ewr',
       'pickup_near_lga', 'dropoff_near_lga', 'pickup_near_esm',
       'dropoff_near_esm', 'pickup_near_ts', 'dropoff_near_ts'],
      dtype='object')


$Fare\_Amount$ Column is in our Data, so it is shifted to the last and is not Scaled by the Pipeline.

In [42]:
X_train = train_data[:,:-1]
y_train = train_data[:,-1]

In [43]:
X_train.shape, y_train.shape

((780465, 19), (780465,))

Transform the Test Data

In [44]:
test_data = pre_pipeline.transform(test_data)

Current Columns: Index(['fare_amount', 'pickup_longitude', 'dropoff_longitude', 'year',
       'H_Distance', 'jfk_Distance_Pickup', 'jfk_Distance_Dropoff',
       'sol_Distance_Dropoff', 'cp_Distance_Pickup', 'cp_Distance_Dropoff',
       'pickup_near_jfk', 'dropoff_near_jfk', 'dropoff_near_ewr',
       'pickup_near_lga', 'dropoff_near_lga', 'pickup_near_esm',
       'dropoff_near_esm', 'pickup_near_ts', 'dropoff_near_ts'],
      dtype='object')


In [45]:
X_test = test_data[:,:-1]
y_test = test_data[:,-1]

In [46]:
X_test.shape, y_test.shape

((195047, 19), (195047,))

### 10-Fold Cross Validation Algorithm

For manual Implementation of Cross Validation, I have implemented the **Repeated Random Sub Sampling CV** method.

It is more robust than the K-fold standard method.

Reference: [Neptune AI](https://neptune.ai/blog/cross-validation-in-machine-learning-how-to-do-it-right)

In [23]:
# Import MSE, R2, MAE, RMSE from sklearn.metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Import Train Test Split from sklearn.model_selection
from sklearn.model_selection import train_test_split

# Manual Implementation of Cross Validation
def cross_validate(model, X, y, cv=5, isEarlyStoppingUsingVal=False):
    rmse = []
    mae = []
    r2 = []
    for i in range(cv):
        # Split X and y into train and Validation sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=i)
        if not isEarlyStoppingUsingVal:
            # Fit the model to the train data
            model.fit(X_train, y_train)
        else:
            # Fit the model but also pass in the validation data for early stopping (This is Optimization Based Model)
            model.fit(X_train, y_train, X_val, y_val)
        # Predict on the validation data
        y_pred = model.predict(X_val)
        # Calculate the score of the model on the validation data
        rmse.append(np.sqrt(mean_squared_error(y_val, y_pred)))
        mae.append(mean_absolute_error(y_val, y_pred))
        r2.append(r2_score(y_val, y_pred))
    return {'rmse': np.array(rmse), 'mae': np.array(mae), 'r2': np.array(r2)}


Function to Beautifully Display the Ouput of Cross-Validation

In [24]:
def print_scores(scores, rmse = True, mae = True, r2 = True):
    rmse_scores = scores['rmse']
    mae_scores = scores['mae']
    r2_scores = scores['r2']
    
    if rmse:
        # Print RMSE Scores
        print("*************************")
        print('RMSE Scores:')
        print("*************************")
        print("Scores: {}".format(rmse_scores))
        print("Mean: {}".format(rmse_scores.mean()))
        print("Std: {}".format(rmse_scores.std()))
        print("*************************")
        print()

    if mae:
        # Print MAE Scores
        print("*************************")
        print('MAE Scores:')
        print("*************************")
        print("Scores: {}".format(mae_scores))
        print("Mean: {}".format(mae_scores.mean()))
        print("Std: {}".format(mae_scores.std()))
        print("*************************")
        print()
    
    if r2:
        # Print R2 Scores
        print("*************************")
        print('R2 Scores:')
        print("*************************")
        print("Scores: {}".format(r2_scores))
        print("Mean: {}".format(r2_scores.mean()))
        print("Std: {}".format(r2_scores.std()))
        print("*************************")
        

Variable to store all scores of all models

In [30]:
total_scores = {}

### 1. Trying Matrix Based Linear Regression Solutions

#### (i)    $w_{estimate} = (A^{T}A)^{-1}A^{T}y_{real}$

In [31]:
class SimpleLinearRegression:
    
    def __init__(self):
        pass
    
    def fit(self, X, y):
        self.w_estimate = np.linalg.inv(X.T @ X) @ X.T @ y
        self.n = X.shape[0]
        self.d = X.shape[1]
        return self
    
    def predict(self, X):
        if self.d != X.shape[1]:
            raise ValueError('Dimension of X does not match dimension of model expected: {}'.format((self.d)))
        return X @ self.w_estimate

#### Cross Validate This Approach on the Training Data

In [32]:
simple_linear_regression = SimpleLinearRegression()
slr_scores = cross_validate(simple_linear_regression, X_train, y_train, cv=10)

In [33]:
print_scores(slr_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.04330889 4.81932008 4.58737578 4.77733321 4.71654854 4.94277339
 4.82211913 4.97569076 4.66094894 4.82120803]
Mean: 4.816662676483373
Std: 0.134539487583856
*************************



In [34]:
total_scores['Simple Linear Regression'] = slr_scores.copy()

#### (ii) $w_{estimate} = A^{⊕}y_{real}$

In [35]:
class PseudoInverseLinearRegression:
    
    def __init__(self):
        pass
    
    def fit(self, X, y):
        # Pseudo Inverse of A
        self.w_estimate = np.linalg.pinv(X) @ y
        self.n = X.shape[0]
        self.d = X.shape[1]
        return self
    
    def predict(self, X):
        if self.d != X.shape[1]:
            raise ValueError('Dimension of X does not match dimension of model expected: {}'.format((self.d)))
        return X @ self.w_estimate

#### Cross Validate This Approach on the Training Data

In [43]:
pseudo_linear_regression = PseudoInverseLinearRegression()
plr_scores = cross_validate(pseudo_linear_regression, X_train, y_train, cv=10)

In [44]:
print_scores(plr_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.04330889 4.81932008 4.58737578 4.77733321 4.71654854 4.94277339
 4.82211913 4.97569076 4.66094894 4.82120803]
Mean: 4.816662676483373
Std: 0.1345394875838561
*************************



In [45]:
total_scores['Pseudo Inverse Linear Regression'] = plr_scores.copy()

#### (iii) $w_{estimate} = R^{-1}Q^{T}y_{real}$

In [39]:
class QRLinearRegression:
    
    def __init__(self):
        pass
    
    def fit(self, X, y):
        Q, R = np.linalg.qr(X)
        self.w_estimate = np.linalg.inv(R) @ Q.T @ y
        self.n = X.shape[0]
        self.d = X.shape[1]
        return self
    
    def predict(self, X):
        if self.d != X.shape[1]:
            raise ValueError('Dimension of X does not match dimension of model expected: {}'.format((self.d)))
        return X @ self.w_estimate

#### Cross Validate This Approach on the Training Data

In [40]:
qr_regression = QRLinearRegression()
qr_scores = cross_validate(qr_regression, X_train, y_train, cv=10)

In [41]:
print_scores(qr_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.04330889 4.81932008 4.58737578 4.77733321 4.71654854 4.94277339
 4.82211913 4.97569076 4.66094894 4.82120803]
Mean: 4.816662676483373
Std: 0.13453948758385617
*************************



In [42]:
total_scores['QR Linear Regression'] = qr_scores.copy()

#### (iv) Simple Linear Regression using Scikit-Learn

In [46]:
from sklearn.linear_model import LinearRegression

#### Cross Validate This Approach on the Training Data

In [47]:
sk_slr_regression = LinearRegression()
sk_slr_scores = cross_validate(sk_slr_regression, X_train, y_train, cv=10)

In [48]:
print_scores(sk_slr_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.04330889 4.81932008 4.58737578 4.77733321 4.71654854 4.94277339
 4.82211913 4.97569076 4.66094894 4.82120803]
Mean: 4.816662676483373
Std: 0.13453948758385603
*************************



In [49]:
total_scores['Sklearn Simple Linear Regression'] = sk_slr_scores.copy()

#### (v) Ridge Regression using Scikit-Learn

In [50]:
from sklearn.linear_model import Ridge

We will Tune the Hyper-Parameter $\lambda$ for Ridge Regression

Where $\lambda$ is the Regularization Parameter

In [51]:
lambdas = [0.0001, 0.001,0.01, 0.1, 1, 10]

There are different solvers for the Ridge Regression, but we will stick with $cholesky$ for now as we are considering matrix based solutions and it is the one taught in the class.

In [52]:
solver = 'cholesky'
seed = 42

***Hyper-Parameter Tuning***

In [53]:
l_tuning_scores = {}
for l in lambdas:
    sk_ridge_regression = Ridge(alpha=l, solver=solver, random_state=seed)
    sk_ridge_scores = cross_validate(sk_ridge_regression, X_train, y_train, cv=10)
    l_tuning_scores[l] = sk_ridge_scores.copy()

Checking Scores for each lambda

In [54]:
for l in lambdas:
    print("Lambda Value:", l)
    print("Mean RMSE Score:", l_tuning_scores[l]['rmse'].mean())
    print("Std. in RMSE Score:", l_tuning_scores[l]['rmse'].std())

Lambda Value: 0.0001
Mean RMSE Score: 4.816662676471172
Std. in RMSE Score: 0.13453948756921277
Lambda Value: 0.001
Mean RMSE Score: 4.816662676361256
Std. in RMSE Score: 0.13453948743744293
Lambda Value: 0.01
Mean RMSE Score: 4.81666267526208
Std. in RMSE Score: 0.13453948611974564
Lambda Value: 0.1
Mean RMSE Score: 4.816662664270679
Std. in RMSE Score: 0.13453947294276966
Lambda Value: 1
Mean RMSE Score: 4.816662554391599
Std. in RMSE Score: 0.13453934117286698
Lambda Value: 10
Mean RMSE Score: 4.816661459092447
Std. in RMSE Score: 0.13453802345935117


Scores for each $\lambda$ seem to be pretty much the same, so we will stick with $\lambda = 1$

In [55]:
sk_ridge_scores = l_tuning_scores[1].copy()

In [56]:
print_scores(sk_ridge_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.04330843 4.81931909 4.58737613 4.77733358 4.71654882 4.94277357
 4.82211893 4.97569041 4.66094851 4.82120809]
Mean: 4.816662554391599
Std: 0.13453934117286698
*************************



In [57]:
total_scores['Sklearn Ridge Regression'] = sk_ridge_scores.copy()

#### (vi) Lasso Regression using Scikit-Learn

In [58]:
from sklearn.linear_model import Lasso

We will Tune the Hyper-Parameter $\lambda$ for Lasso Regression

Where $\lambda$ is the Regularization Parameter

In [59]:
lambdas = [0.001, 0.01, 0.1, 1, 10]

***Hyper-Parameter Tuning***

In [60]:
l_tuning_scores_lasso = {}
for l in lambdas[::-1]:
    print("Lambda:", l)
    sk_lasso_regression = Lasso(alpha=l, max_iter=10000, random_state=seed)
    sk_lasso_scores = cross_validate(sk_lasso_regression, X_train, y_train, cv=10)
    l_tuning_scores_lasso[l] = sk_lasso_scores.copy()

Lambda: 10
Lambda: 1
Lambda: 0.1
Lambda: 0.01
Lambda: 0.001


Checking Scores for each lambda

In [61]:
for l in lambdas:
    print("Lambda Value:", l)
    print("Mean RMSE Score:", l_tuning_scores_lasso[l]['rmse'].mean())
    print("Std. in RMSE Score:", l_tuning_scores_lasso[l]['rmse'].std())

Lambda Value: 0.001
Mean RMSE Score: 4.816701142384302
Std. in RMSE Score: 0.1345366670079097
Lambda Value: 0.01
Mean RMSE Score: 4.81763826703508
Std. in RMSE Score: 0.13450774871143759
Lambda Value: 0.1
Mean RMSE Score: 4.869981452707974
Std. in RMSE Score: 0.1359601122825223
Lambda Value: 1
Mean RMSE Score: 5.41208578733203
Std. in RMSE Score: 0.1376848113477227
Lambda Value: 10
Mean RMSE Score: 9.75093243901511
Std. in RMSE Score: 0.07505701007643602


Scores for each $\lambda$ seem to increase as we decrease the $\lambda$. We will stick with $\lambda = 0.001$

In [62]:
sk_lasso_scores = l_tuning_scores_lasso[0.001]

In [63]:
print_scores(sk_lasso_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.04323604 4.81915949 4.58744961 4.77733444 4.71682149 4.94291219
 4.82215445 4.97575332 4.66076459 4.82142579]
Mean: 4.816701142384302
Std: 0.1345366670079097
*************************



In [64]:
total_scores['Sklearn Lasso Regression'] = sk_lasso_scores.copy()

#### (vii) Elastic Net Regression using Scikit-Learn

In [65]:
from sklearn.linear_model import ElasticNet

For Elastic Net, we will Tune two Hyper-Parameters:

1. $\lambda$ i.e. Regularization Parameter
2. $ll\_ratio$ i.e. Mix Ratio of $L1$ and $L2$ Regularization

$ll\_ratio = 1$ means Lasso Regression i.e. only $L1$ Regularization <br>
$ll\_ratio = 0$ means Ridge Regression i.e. only $L2$ Regularization

In [66]:
lambdas = [0.001, 0.01, 0.1, 1, 10]
ll_ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9]

A total of 45 Combinations of $\lambda$ and $ll\_ratio$ are considered for the Hyper-Parameter Tuning

This step takes lots of time, so we will use `Randomized Grid Search` to find the best Hyper-Parameters

Implemented **Randomized Grid** Search to find the best Hyper-Parameters

- We will try 30 different Combinations

In [67]:
import random
l_tuning_scores_elastic = {}

for _ in range(30):
    
    l = random.choice(lambdas)
    ll = random.choice(ll_ratios)
    while (l, ll) in l_tuning_scores_elastic:
        # If a combination is already visited, choose another one
        l = random.choice(lambdas)
        ll = random.choice(ll_ratios)
        
    print(str(_)+".)", "Lambda:", l, "L1 Ratio:", ll)
    sk_elastic_regression = ElasticNet(alpha=l, l1_ratio=ll, max_iter=10000, random_state=seed)
    sk_elastic_scores = cross_validate(sk_elastic_regression, X_train, y_train, cv=10)
    l_tuning_scores_elastic[(l, ll)] = sk_elastic_scores.copy()

0.) Lambda: 0.01 L1 Ratio: 0.7
1.) Lambda: 0.1 L1 Ratio: 0.1
2.) Lambda: 0.001 L1 Ratio: 0.7
3.) Lambda: 10 L1 Ratio: 0.7
4.) Lambda: 1 L1 Ratio: 0.4
5.) Lambda: 0.1 L1 Ratio: 0.7
6.) Lambda: 0.001 L1 Ratio: 0.2
7.) Lambda: 1 L1 Ratio: 0.7
8.) Lambda: 1 L1 Ratio: 0.5
9.) Lambda: 0.1 L1 Ratio: 0.4
10.) Lambda: 0.01 L1 Ratio: 0.8
11.) Lambda: 1 L1 Ratio: 0.1
12.) Lambda: 0.01 L1 Ratio: 0.4
13.) Lambda: 1 L1 Ratio: 0.8
14.) Lambda: 1 L1 Ratio: 0.3
15.) Lambda: 0.001 L1 Ratio: 0.5
16.) Lambda: 10 L1 Ratio: 0.8
17.) Lambda: 0.1 L1 Ratio: 0.3
18.) Lambda: 10 L1 Ratio: 0.2
19.) Lambda: 0.001 L1 Ratio: 0.4
20.) Lambda: 0.01 L1 Ratio: 0.2
21.) Lambda: 0.1 L1 Ratio: 0.5
22.) Lambda: 0.001 L1 Ratio: 0.9
23.) Lambda: 0.01 L1 Ratio: 0.5
24.) Lambda: 10 L1 Ratio: 0.4
25.) Lambda: 1 L1 Ratio: 0.9
26.) Lambda: 0.001 L1 Ratio: 0.1
27.) Lambda: 0.01 L1 Ratio: 0.3
28.) Lambda: 10 L1 Ratio: 0.3
29.) Lambda: 0.1 L1 Ratio: 0.8


Let's find the best parameters out of these!

In [68]:
# We will select the combination with the lowest Average RMSE Score
min_avg_rmse = float('inf')
std_rmse = float('inf')
min_avg_rmse_l = None

for (l, ll) in l_tuning_scores_elastic:
    if l_tuning_scores_elastic[(l, ll)]['rmse'].mean() < min_avg_rmse:
        min_avg_rmse = l_tuning_scores_elastic[(l, ll)]['rmse'].mean()
        std_rmse = l_tuning_scores_elastic[(l, ll)]['rmse'].std()
        min_avg_rmse_l = (l, ll)

In [69]:
print("The Best Combination is:", min_avg_rmse_l, "with an Average RMSE Score of:", min_avg_rmse, "and a Std. Dev. of:", std_rmse)

The Best Combination is: (0.001, 0.1) with an Average RMSE Score of: 4.81661063548957 and a Std. Dev. of: 0.13445685286865294


In [70]:
sk_elastic_scores = l_tuning_scores_elastic[min_avg_rmse_l]

In [71]:
print_scores(sk_elastic_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.0430546  4.81876002 4.5875908  4.7775493  4.71674455 4.94289333
 4.82203123 4.97550987 4.66069899 4.82127366]
Mean: 4.81661063548957
Std: 0.13445685286865294
*************************



In [72]:
total_scores['Sklearn Elastic Net Regression'] = sk_elastic_scores.copy()

#### (viii) Manually Implemented Ridge Regression (Closed Form Solution)

### $w_{estimate} = (A^{T}A + \lambda I_{d+1})^{-1}A^{T}y_{real}$

Where $A$ is the Input Matrix with ones column added, $y_{real}$ is the Real Output, $\lambda$ is the Regularization Parameter and $I_{d+1}$ is the Identity Matrix with $d+1$ rows and $d+1$ columns

In [26]:
class RidgeRegression:
    
    def __init__(self, lambda_=0.1):
        self.lambda_ = lambda_
    
    def fit(self, X, y):
        self.n = X.shape[0]
        self.d = X.shape[1]
        self.w_estimate = np.linalg.inv(X.T @ X + self.lambda_ * np.eye(self.d)) @ X.T @ y
        return self
    
    def predict(self, X):
        if self.d != X.shape[1]:
            raise ValueError('Dimension of X does not match dimension of model expected: {}'.format((self.d)))
        return X @ self.w_estimate

We will Tune the Hyper-Parameter $\lambda$ for Ridge Regression

Where $\lambda$ is the Regularization Parameter

In [74]:
lambdas = [0.0001, 0.001,0.01, 0.1, 1, 10]

***Hyper-Parameter Tuning***

In [75]:
ridge_tuning_scores = {}
for l in lambdas:
    ridge_regression = RidgeRegression(lambda_=l)
    ridge_scores = cross_validate(ridge_regression, X_train, y_train, cv=10)
    ridge_tuning_scores[l] = ridge_scores.copy()

Checking Scores for each lambda

In [76]:
for l in lambdas:
    print("Lambda Value:", l)
    print("Mean RMSE Score:", l_tuning_scores[l]['rmse'].mean())
    print("Std. in RMSE Score:", l_tuning_scores[l]['rmse'].std())

Lambda Value: 0.0001
Mean RMSE Score: 4.816662676471172
Std. in RMSE Score: 0.13453948756921277
Lambda Value: 0.001
Mean RMSE Score: 4.816662676361256
Std. in RMSE Score: 0.13453948743744293
Lambda Value: 0.01
Mean RMSE Score: 4.81666267526208
Std. in RMSE Score: 0.13453948611974564
Lambda Value: 0.1
Mean RMSE Score: 4.816662664270679
Std. in RMSE Score: 0.13453947294276966
Lambda Value: 1
Mean RMSE Score: 4.816662554391599
Std. in RMSE Score: 0.13453934117286698
Lambda Value: 10
Mean RMSE Score: 4.816661459092447
Std. in RMSE Score: 0.13453802345935117


Scores for each $\lambda$ seem to be pretty much the same, so we will stick with $\lambda = 1$

In [77]:
ridge_scores = l_tuning_scores[1].copy()

In [78]:
print_scores(ridge_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.04330843 4.81931909 4.58737613 4.77733358 4.71654882 4.94277357
 4.82211893 4.97569041 4.66094851 4.82120809]
Mean: 4.816662554391599
Std: 0.13453934117286698
*************************



In [79]:
total_scores['Ridge Regression Manual'] = ridge_scores.copy()

Saving the Net Accuracies so far

In [81]:
# import pickle

# with open('total_scores_11940140.pickle', 'wb') as f:
#     pickle.dump(total_scores, f)

### 2. Trying Optimization Based Linear Regression Solutions

![SGD](image_sgd.png)

### $\theta_{new} = \theta - \eta \times \nabla_{\theta}MSE(\theta)$ 

#### (i) Batch Gradient Descent

In [54]:
# Import MSE
from sklearn.metrics import mean_squared_error

class BatchGradientRegressor:
    
    def __init__(self, learning_rate=0.1, n_iters=1000, seed=42, error_threshold=1e-6, convergence_threshold=1e-6):
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.error_threshold = error_threshold
        self.convergence_threshold = convergence_threshold
        self.seed = seed
    
    def calculate_gradient(self, X, y, w):
        """
        Calculate the gradient of the loss function with respect to w.
        """
        return (2 / len(y)) * X.T @ (X @ w - y)
    
    # X_val and y_val are the validation sets used for early stopping
    def fit(self, X, y, X_val, y_val):
        
        self.d = X.shape[1]
        
        # Randomise an Initial Theta with seed = self.seed
        np.random.seed(self.seed)
        theta = np.random.randn(self.d)
        
        # Iterate
        for _ in range(self.n_iters):
            
            # Calculate the Gradient
            grad = self.calculate_gradient(X, y, theta)
            # Calculate New Theta
            theta_new = theta - self.learning_rate * grad
            
            # Check Convergence
            if np.linalg.norm(theta_new - theta) < self.convergence_threshold:
                theta = theta_new
                break
            
            # Check MSE for Early Stopping
            y_pred = X_val @ theta_new
            mse = mean_squared_error(y_pred, y_val)
            if mse < self.error_threshold:
                theta = theta_new
                break
            
            # Update Theta
            theta = theta_new.copy()
            
        self.theta = theta.copy()
        return self

    def predict(self, X):
        """
        Predict the output of a dataset X.
        """
        if X.shape[1] != self.d:
            raise ValueError("X has wrong number of features, expected {}, got {}.".format(self.d, X.shape[1]))
        return X @ self.theta

We will Tune the parameter $\eta$ for Batch Gradient Descent

Where $\eta$ is the Learning Rate

In [83]:
eta_values = [0.01, 0.1, 0.5, 1, 2]

Hyper-Parameter Tuning

In [84]:
b_tuning_scores_batch = {}

for eta in eta_values:
    print("Eta Value:", eta)
    batch_gradient_regression = BatchGradientRegressor(learning_rate=eta, n_iters=100, seed=42)
    batch_gradient_scores = cross_validate(batch_gradient_regression, X_train, y_train, cv=10, isEarlyStoppingUsingVal=True)
    b_tuning_scores_batch[eta] = batch_gradient_scores.copy()

Eta Value: 0.01
Eta Value: 0.1
Eta Value: 0.5
Eta Value: 1
Eta Value: 2


Let's find the best value for $\eta$

In [85]:
min_rmse = float('inf')
min_rmse_eta = None
min_rmse_std = None

for eta in b_tuning_scores_batch:
    if b_tuning_scores_batch[eta]['rmse'].mean() < min_rmse:
        min_rmse = b_tuning_scores_batch[eta]['rmse'].mean()
        min_rmse_eta = eta
        min_rmse_std = b_tuning_scores_batch[eta]['rmse'].std()

In [86]:
print("Best Eta Value is:", min_rmse_eta, "with an Average RMSE Score of:", min_rmse, "and a Std. Dev. of:", min_rmse_std)

Best Eta Value is: 0.1 with an Average RMSE Score of: 4.815670541368664 and a Std. Dev. of: 0.13473601004127464


In [87]:
batch_gradient_regression_scores = b_tuning_scores_batch[min_rmse_eta]
print_scores(batch_gradient_regression_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.0414098  4.82100321 4.58776729 4.77704287 4.71286819 4.94371966
 4.81358413 4.97697486 4.66045338 4.82188203]
Mean: 4.815670541368664
Std: 0.13473601004127464
*************************



In [88]:
total_scores['Batch Gradient Regression'] = batch_gradient_regression_scores.copy()

#### (ii) Stochastic Gradient Descent

In [89]:
# Import MSE
from sklearn.metrics import mean_squared_error

class StochasticGradientRegressor:
    
    def __init__(self, initial_learning_rate=0.1, n_epochs=100, seed=42, error_threshold=1e-6, convergence_threshold=1e-9, k=0.1):
        self.initial_learning_rate = initial_learning_rate
        self.n_epochs = n_epochs
        self.error_threshold = error_threshold
        self.convergence_threshold = convergence_threshold
        self.k = k
        self.seed = seed
    
    def calculate_gradient(self, X, y, w):
        """
        Calculate the gradient of the loss function with respect to w.
        """
        return (2 / len(y)) * X.T @ (X @ w - y)
    
    def exponential_learning_schedule(self, t):
        """
        Exponential Learning Schedule.
        """
        return self.initial_learning_rate * np.exp(-t * self.k)
    
    # X_val and y_val are the validation sets used for early stopping
    def fit(self, X, y, X_val, y_val):
        
        self.d = X.shape[1]
        
        # Randomise an Initial Theta with seed = self.seed
        np.random.seed(self.seed)
        theta = np.random.randn(self.d)
        
        # Iterate
        t = 0
        flag = False
        for _ in range(self.n_epochs):
            
            for i in range(len(y)):
                
                # Calculate the Gradient
                grad = self.calculate_gradient(np.array([X[i]]), np.array([y[i]]), theta)
                # Calculate New Theta
                theta_new = theta - self.exponential_learning_schedule(t=t) * grad
                
                # Check Convergence
                if np.linalg.norm(theta_new - theta) < self.convergence_threshold:
                    theta = theta_new
                    flag = True
                    break
                
                # Update Theta
                theta = theta_new.copy()
                t += 1
            
            # Check MSE for Early Stopping after each epoch
            y_pred = X_val @ theta
            mse = mean_squared_error(y_pred, y_val)
            if mse < self.error_threshold:
                theta = theta_new
                flag = True
                break
            
            # Checking for Convergence or Early Stopping
            if flag:
                break
            
        self.theta = theta.copy()
        return self

    def predict(self, X):
        """
        Predict the output of a dataset X.
        """
        if X.shape[1] != self.d:
            raise ValueError("X has wrong number of features, expected {}, got {}.".format(self.d, X.shape[1]))
        return X @ self.theta

For the Above Gradient Descent, We have selected an Exponential Decay Function for Learning Rate $\eta$

$$\eta_{new} = \eta_{initial} \times \exp(-k * t)$$

where $k$ is the Learning Rate Decay Factor and $t$ is the Current Iteration Number

Hence there are two hyper-parameters to tune:

1. $\eta$ i.e. Learning Rate
2. $k$ i.e. Decay Rate

In [90]:
eta_values = [0.01, 0.1, 0.5, 1, 2, 5]
k_values = [0.1, 0.5, 1, 2, 5]

A Total of 30 Combinations of $\eta$ and $k$ are considered for the Hyper-Parameter Tuning

Hyper-Parameter Tuning

In [91]:
stochastic_tuning_scores = {}

for eta in eta_values:
    for k in k_values:
        print("Eta Value:", eta, "K Value:", k)
        stochastic_gradient_regression = StochasticGradientRegressor(initial_learning_rate=eta, n_epochs=500, seed=42, error_threshold=1e-6, convergence_threshold=1e-6, k=k)
        stochastic_gradient_scores = cross_validate(stochastic_gradient_regression, X_train, y_train, cv=10, isEarlyStoppingUsingVal=True)
        stochastic_tuning_scores[(eta, k)] = stochastic_gradient_scores.copy()

Eta Value: 0.01 K Value: 0.1
Eta Value: 0.01 K Value: 0.5
Eta Value: 0.01 K Value: 1
Eta Value: 0.01 K Value: 2
Eta Value: 0.01 K Value: 5
Eta Value: 0.1 K Value: 0.1
Eta Value: 0.1 K Value: 0.5
Eta Value: 0.1 K Value: 1
Eta Value: 0.1 K Value: 2
Eta Value: 0.1 K Value: 5
Eta Value: 0.5 K Value: 0.1
Eta Value: 0.5 K Value: 0.5
Eta Value: 0.5 K Value: 1
Eta Value: 0.5 K Value: 2
Eta Value: 0.5 K Value: 5
Eta Value: 1 K Value: 0.1
Eta Value: 1 K Value: 0.5
Eta Value: 1 K Value: 1
Eta Value: 1 K Value: 2
Eta Value: 1 K Value: 5
Eta Value: 2 K Value: 0.1
Eta Value: 2 K Value: 0.5
Eta Value: 2 K Value: 1
Eta Value: 2 K Value: 2
Eta Value: 2 K Value: 5
Eta Value: 5 K Value: 0.1
Eta Value: 5 K Value: 0.5
Eta Value: 5 K Value: 1
Eta Value: 5 K Value: 2
Eta Value: 5 K Value: 5


Let's find the best parameters out of these!

In [93]:
# Find the Best Learning Rate and K Value
best_eta = float("-inf")
best_k = float("-inf")
min_rmse = float("inf")
std_of_min_rmse = float("inf")

for eta in eta_values:
    for k in k_values:
        if stochastic_tuning_scores[(eta, k)]["rmse"].mean() < min_rmse:
            best_eta = eta
            best_k = k
            min_rmse = stochastic_tuning_scores[(eta, k)]["rmse"].mean()
            std_of_min_rmse = stochastic_tuning_scores[(eta, k)]["rmse"].std()

In [94]:
print("Best Learning Rate:", best_eta)
print("Best K Value:", best_k)
print("Minimum RMSE:", min_rmse)
print("Standard Deviation of Minimum RMSE:", std_of_min_rmse)

Best Learning Rate: 0.01
Best K Value: 0.1
Minimum RMSE: 14.240562330964224
Standard Deviation of Minimum RMSE: 1.8524472549958861


In [95]:
stochastic_gradient_regression_scores = stochastic_tuning_scores[(best_eta, best_k)]
print_scores(stochastic_gradient_regression_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [16.95945805 17.66174809 15.16425607 13.30838709 11.8275381  14.34231345
 14.53172087 11.86976031 13.84087845 12.89956283]
Mean: 14.240562330964224
Std: 1.8524472549958861
*************************



In [96]:
total_scores['Stochastic Gradient Regression'] = stochastic_gradient_regression_scores.copy()

#### (iii) Mini-Batch Stochastic Gradient Descent

In [97]:
# Import MSE
from sklearn.metrics import mean_squared_error

class MiniBatchGradientRegressor:
    
    def __init__(self, initial_learning_rate=0.1, n_epochs=100, seed=42, error_threshold=1e-6, convergence_threshold=1e-9, k=0.1, mini_batch_size=32):
        self.initial_learning_rate = initial_learning_rate
        self.n_epochs = n_epochs
        self.error_threshold = error_threshold
        self.convergence_threshold = convergence_threshold
        self.mini_batch_size = mini_batch_size
        self.k = k
        self.seed = seed
    
    def calculate_gradient(self, X, y, w):
        """
        Calculate the gradient of the loss function with respect to w.
        """
        # Print Shapes
        # print(X.shape, y.shape, w.shape, (X.T @ ((X @ w) - y)).shape)
        
        return (2 / len(y)) * X.T @ ((X @ w) - y)
    
    def exponential_learning_schedule(self, t):
        """
        Exponential Learning Schedule.
        """
        return self.initial_learning_rate * np.exp(-t * self.k)
    
    # X_val and y_val are the validation sets used for early stopping
    def fit(self, X, y, X_val, y_val):
        
        self.d = X.shape[1]
        
        # Randomise an Initial Theta with seed = self.seed
        np.random.seed(self.seed)
        theta = np.random.randn(self.d)
        
        # Iterate
        t = 0
        flag = False
        for _ in range(self.n_epochs):
            
            shuffled_indices = np.random.permutation(len(y))
            X_shuffled = X[shuffled_indices].copy()
            y_shuffled = y[shuffled_indices].copy()
            
            for i in range(0, len(y), self.mini_batch_size):
                
                X_batch = X_shuffled[i:i + self.mini_batch_size].copy()
                y_batch = y_shuffled[i:i + self.mini_batch_size].copy()
            
                # Calculate the Gradient
                grad = self.calculate_gradient(X_batch, y_batch, theta)
                
                # Calculate New Theta
                theta_new = theta - self.exponential_learning_schedule(t=t) * grad
                
                # Check Convergence
                if np.linalg.norm(theta_new - theta) < self.convergence_threshold:
                    theta = theta_new
                    flag = True
                    break
                
                # Update Theta
                theta = theta_new.copy()
                t += 1
            
            # Check MSE for Early Stopping after each epoch
            y_pred = X_val @ theta
            mse = mean_squared_error(y_pred, y_val)
            if mse < self.error_threshold:
                theta = theta_new
                flag = True
                break
            
            # Checking for Convergence or Early Stopping
            if flag:
                break
            
        self.theta = theta.copy()
        return self

    def predict(self, X):
        """
        Predict the output of a dataset X.
        """
        if X.shape[1] != self.d:
            raise ValueError("X has wrong number of features, expected {}, got {}.".format(self.d, X.shape[1]))
        return X @ self.theta

For the Above Gradient Descent, We have selected an Exponential Decay Function for Learning Rate $\eta$

$$\eta_{new} = \eta_{initial} \times \exp(-k * t)$$

where $k$ is the Learning Rate Decay Factor and $t$ is the Current Iteration Number

Hence there are three hyper-parameters to tune:

1. $\eta$ i.e. Learning Rate
2. $k$ i.e. Decay Rate
3. $mini\_batch\_size$ i.e. Batch Size

In [98]:
eta_values = [0.01, 0.1, 0.5, 1, 2, 5]
k_values = [0.1, 0.5, 1, 2, 5]
mini_batch_size_values = [32, 64, 128, 256, 512, 1024]

A Total of 180 Combinations of $\eta$, $k$ and $mini\_batch\_size$ are possible for the Hyper-Parameter Tuning

So we will apply `Randomized Grid Search` to find the best Hyper-Parameters and try for 30 Random Combinations

Hyper-Parameter Tuning

In [99]:
minibatch_tuning_scores = {}

for _ in range(30):
    
    eta = np.random.choice(eta_values)
    k = np.random.choice(k_values)
    mini_batch_size = np.random.choice(mini_batch_size_values)
    
    while (eta, k, mini_batch_size) in minibatch_tuning_scores:
        eta = np.random.choice(eta_values)
        k = np.random.choice(k_values)
        mini_batch_size = np.random.choice(mini_batch_size_values)
        
    print("Eta:", eta, "K:", k, "Mini Batch Size:", mini_batch_size)
    minibatch_regressor = MiniBatchGradientRegressor(initial_learning_rate=eta, k=k, mini_batch_size=mini_batch_size, n_epochs=500)
    # Cross Validation
    cross_validated_scores = cross_validate(minibatch_regressor, X_train, y_train, cv=10, isEarlyStoppingUsingVal=True)
    minibatch_tuning_scores[(eta, k, mini_batch_size)] = cross_validated_scores.copy()

Eta: 5.0 K: 1.0 Mini Batch Size: 256
Eta: 0.1 K: 1.0 Mini Batch Size: 1024
Eta: 2.0 K: 0.1 Mini Batch Size: 128
Eta: 0.5 K: 0.1 Mini Batch Size: 1024
Eta: 0.01 K: 5.0 Mini Batch Size: 512
Eta: 2.0 K: 0.1 Mini Batch Size: 32
Eta: 0.01 K: 5.0 Mini Batch Size: 256
Eta: 5.0 K: 2.0 Mini Batch Size: 1024
Eta: 1.0 K: 5.0 Mini Batch Size: 32
Eta: 0.5 K: 5.0 Mini Batch Size: 256
Eta: 2.0 K: 0.5 Mini Batch Size: 32
Eta: 2.0 K: 1.0 Mini Batch Size: 256
Eta: 1.0 K: 0.1 Mini Batch Size: 1024
Eta: 0.5 K: 1.0 Mini Batch Size: 256
Eta: 0.1 K: 5.0 Mini Batch Size: 128
Eta: 0.5 K: 1.0 Mini Batch Size: 1024
Eta: 0.01 K: 5.0 Mini Batch Size: 32
Eta: 0.5 K: 5.0 Mini Batch Size: 128
Eta: 2.0 K: 2.0 Mini Batch Size: 512
Eta: 1.0 K: 1.0 Mini Batch Size: 512
Eta: 5.0 K: 5.0 Mini Batch Size: 512
Eta: 0.1 K: 0.1 Mini Batch Size: 1024
Eta: 2.0 K: 0.1 Mini Batch Size: 1024
Eta: 0.01 K: 0.1 Mini Batch Size: 512
Eta: 0.01 K: 5.0 Mini Batch Size: 128
Eta: 0.1 K: 1.0 Mini Batch Size: 256
Eta: 0.5 K: 0.1 Mini Batch Siz

Let's find the best parameters out of these!

In [100]:
# Find the Best Learning Rate and K Value
best_eta = float("-inf")
best_k = float("-inf")
best_mini_batch_size = float("-inf")
min_rmse = float("inf")
std_of_min_rmse = float("inf")

for (eta, k, mbs) in minibatch_tuning_scores:
    if minibatch_tuning_scores[(eta, k, mbs)]["rmse"].mean() < min_rmse:
        min_rmse = minibatch_tuning_scores[(eta, k, mbs)]["rmse"].mean()
        std_of_min_rmse = minibatch_tuning_scores[(eta, k, mbs)]["rmse"].std()
        best_eta = eta
        best_k = k
        best_mini_batch_size = mbs

In [101]:
print("Best Learning Rate:", best_eta)
print("Best K Value:", best_k)
print("Best Mini Batch Size:", best_mini_batch_size)
print("Minimum RMSE:", min_rmse)
print("Standard Deviation of Minimum RMSE:", std_of_min_rmse)

Best Learning Rate: 0.1
Best K Value: 0.1
Best Mini Batch Size: 1024
Minimum RMSE: 5.228997375114249
Standard Deviation of Minimum RMSE: 0.11715114863382442


In [102]:
minibatch_regression_scores = minibatch_tuning_scores[(best_eta, best_k, best_mini_batch_size)]
print_scores(minibatch_regression_scores, rmse=True, mae=False, r2=False)

*************************
RMSE Scores:
*************************
Scores: [5.41859187 5.29402914 5.10223908 5.11786904 5.08712505 5.34181552
 5.24732494 5.34933577 5.08521063 5.24643272]
Mean: 5.228997375114249
Std: 0.11715114863382442
*************************



In [103]:
total_scores["Mini-Batch Regression"] = minibatch_regression_scores.copy()

### 3. Trying Non-Parameteric Approach (K-Nearest Neighbors)

In [104]:
class KNN:
    
    def __init__(self, k=3, merge_function='mean', distances='eucledian'):
        self.k = k
        self.distances = distances
        self.merge_function = merge_function
        
    def fit(self, X, y):
        self.X = X.copy()
        self.y = y.copy()
        self.d = X.shape[1]
        return self

    def cosine_distance(self, x1, x2):
        return 1 - (np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2)))
    
    def eucledian_distance(self, x1, x2):
        return np.linalg.norm(x1 - x2)
    
    def merge(self, X):
        # X -> [(Distance, Y)]
        if self.merge_function == 'mean':
            # Get a mean of all the Y values
            return np.mean(np.array([x[1] for x in X]), axis=0)        
        if self.merge_function == 'median':
            # Get a median of all the Y values
            return np.median(np.array([x[1] for x in X]), axis=0)
        # Naive Weighted Average
        if self.merge_function == 'weighted_mean':
            wm = 0
            w_sum = 0
            for (dist, y) in X:
                wm += (np.exp(-dist)) * y
                w_sum += (np.exp(-dist))
            return wm / w_sum
        
        raise ValueError("Invalid merge function.")

    def predict(self, X):
        # Get Nearest K
        predictions = []
        for x in X:
            all_neighbours = []            
            for i in range(len(self.X)):
                # (Distance, Y Value)
                if self.distances == 'cosine':
                    all_neighbours.append((self.cosine_distance(x, self.X[i]), self.y[i]))
                elif self.distances == 'eucledian':
                    all_neighbours.append((self.eucledian_distance(x, self.X[i]), self.y[i]))
            # Sort by Distance
            all_neighbours.sort(key=lambda x: x[0])
            # Get K Nearest Neighbours
            k_nearest_neighbours = all_neighbours[:self.k]
            # Predict Output based on these
            predictions.append(self.merge(k_nearest_neighbours))
        return np.array(predictions)

For our KNN Approach, we have the following Hyper-Parameters:

1. $k$ i.e. Number of Neighbors
2. $merge\_function$ i.e. Kernel Function which will be used to compute the answer from the the Top-K Neighbor Values
3. $distances$ i.e. Distance Metric

Following are the different ***Merge Functions***:

1. **Mean** - Average of the Top-K Neighbor Values
2. **Median** - Median of the Top-K Neighbor Values
3. **Weighted Mean** - Weighted Average of the Top-K Neighbor Values where Weights are defined as $exp(-d) and $d$ is the distance of that datapoint

Following are the different ***Distance*** Metrics:

1. **Euclidean Distance** : $  d(x,y) = \sqrt{(x_1 - y_1)^2 + (x_2 - y_2)^2 + \cdots + (x_d - y_d)^2}$
2. **Cosine Distance** i.e. **1 - Cosine Similarity** : $d(x,y) = 1 - \frac{\sum_{i=1}^{d}x_i y_i}{\sqrt{\sum_{i=1}^{d}x_i^2} \sqrt{\sum_{i=1}^{d}y_i^2}}$

In [105]:
distances = ['cosine', 'eucledian']
merge_functions = ['mean', 'median', 'weighted_mean', 'e_kernel', 'threshold_kernel']
k_values = [1, 3, 5, 7, 9]

Hyper-Parameter Tuning

In [ ]:
for d in distances:
    for kernel in merge_functions:
        for k in k_values:
            print("KNN:", d, kernel, k)
            knn = KNN(k=k, merge_function=kernel, distances=d)
            knn.fit(X_train, y_train)
            knn_scores = cross_validate(knn, X_train, y_train, cv=10, isEarlyStoppingUsingVal=False)
            print_scores(knn_scores, rmse=True, mae=False, r2=False)
            print("")

This was taking alot of time even for a single combination of parameters. Hence, I am opting for Skleans's KNN Implementation as it is much more optimized.

#### KNN Implementation using Scikit-Learn

In [106]:
# Import KNN Regressor
from sklearn.neighbors import KNeighborsRegressor

Hyper-Parameters:

1. Number of Neighbours
2. Weights: 'uniform', 'distance' or a callable function
3. p: Power parameter for the Minkowski metric (p = 1 for Manhattan distance and p = 2 for Euclidean distance)
4. metric: The distance metric to use for the tree. 
    - 'minkowski' : use the Minkowski metric (the default). Here, p=1 for Manhattan Distance and p=2 for Euclidean Distance.
    - 'cosine' : use the cosine metric.
    - 'haversine' : use the haversine metric.
    - 'cityblock' : use the city block metric.

In [107]:
def e_kernel(distances):
    return np.exp(-distances)

In [108]:
num_neighbours = [1, 3, 5, 7, 9]
weights = ['uniform', e_kernel]
p = [1, 2]
metric = ['minkowski', 'cosine', 'haversine', 'cityblock']

For this set of Hyper-Parameters, we have 80 possible Combinations.

Hence, we will again use `Randomized Grid Search` to find the best Hyper-Parameters

In [ ]:
min_rmse = float("inf")
best_num_neighbours = 0
best_weight = ""
best_p = 0
best_metric = ""
knn_tuning_scores = {}

for _ in range(30):
    
    # Randomly select a number of neighbours, weight and metric
    n = np.random.choice(num_neighbours)
    w = np.random.choice(weights)
    m = np.random.choice(metric)
    p = np.random.choice(p)
    
    while (n, w, m, p) in knn_tuning_scores:
        n = np.random.choice(num_neighbours)
        w = np.random.choice(weights)
        m = np.random.choice(metric)
        p = np.random.choice(p)
    
    print("KNN:", n, w, m, p)
    
    knn = KNeighborsRegressor(n_neighbors=n, weights=w, metric=m, p=p)
    knn.fit(X_train, y_train)
    knn_scores = cross_validate(knn, X_train, y_train, cv=10, isEarlyStoppingUsingVal=False)
    if knn_scores["rmse"].mean() < min_rmse:
        min_rmse = knn_scores["rmse"].mean()
        best_num_neighbours = n
        best_weight = w
        best_p = p
        best_metric = m


Even this is taking so much time.... :(

Hence, we could not perform K-Nearset Neighbors Regression.

### Selecting Best Models for Matrix Based and Optimization Based

In [112]:
for model in total_scores:
    
    print(model, end=': ')
    # Print Average RMSE
    print(total_scores[model]["rmse"].mean())

Simple Linear Regression: 4.816662676483373
Pseudo Inverse Linear Regression: 4.816662676483373
QR Linear Regression: 4.816662676483373
Sklearn Simple Linear Regression: 4.816662676483373
Sklearn Ridge Regression: 4.816662554391599
Sklearn Lasso Regression: 4.816701142384302
Sklearn Elastic Net Regression: 4.81661063548957
Ridge Regression Manual: 4.816662554391599
Batch Gradient Regression: 4.815670541368664
Stochastic Gradient Regression: 14.240562330964224
Mini-Batch Regression: 5.228997375114249


Clearly, in Terms of Accuracy:

- For Matrix Based Models, we have found the best model is Ridge Regression
- For Optimization Based Models, we have found the best model is Batch Gradient Descent

## Training these Models on Complete Training Data and Testing on Test Data

**Model-1 : Ridge Linear Regression**

In the Hyper-Parameter Tuning, we have found the best value for $\lambda$ is 1

Training on Complete Training Data

In [70]:
ridge_linear_regression = RidgeRegression(lambda_=1)

In [71]:
ridge_linear_regression.fit(X_train, y_train)

Predicting for Test Data

In [72]:
y_pred = ridge_linear_regression.predict(X_test)

Accuracy Score on Test Data

In [73]:
# Import mean_squared_error
from sklearn.metrics import mean_squared_error
print("RMSE for Test Data: ", np.sqrt(mean_squared_error(y_test, y_pred)))

RMSE for Test Data:  4.791831611358817


Writing the Final Equation

In [74]:
ridge_linear_regression.w_estimate

array([11.34005581, -1.03627812, -0.42061303,  0.97007229,  6.84581034,
       -1.62324499,  0.45000422,  0.93558349, -1.11857759,  1.5537534 ,
        0.82385253,  0.34212281,  0.85203588,  1.15565897,  0.64775276,
       -0.16681822,  0.17252397, -0.20962758,  0.13267107])

In [75]:
feature_order = feature_order = ['1', 'pickup_longitude', 'dropoff_longitude', 'year',
       'H_Distance', 'jfk_Distance_Pickup', 'jfk_Distance_Dropoff',
       'sol_Distance_Dropoff', 'cp_Distance_Pickup', 'cp_Distance_Dropoff',
       'pickup_near_jfk', 'dropoff_near_jfk', 'dropoff_near_ewr',
       'pickup_near_lga', 'dropoff_near_lga', 'pickup_near_esm',
       'dropoff_near_esm', 'pickup_near_ts', 'dropoff_near_ts']
coefficients = list(ridge_linear_regression.w_estimate)

In [76]:
# Print the Equation of the model
print("Equation of the model: ")

print("y = ", end="")
for i in range(len(coefficients)):

    print(coefficients[i], end=" ")
    print("*", end=" ")
    print(feature_order[i], end=" ")
    print(" ", end="")

    if i < len(coefficients) - 1:
        print("+")

Equation of the model: 
y = 11.340055810742994 * 1  +
-1.036278122588903 * pickup_longitude  +
-0.4206130339446861 * dropoff_longitude  +
0.9700722933149348 * year  +
6.8458103387280795 * H_Distance  +
-1.6232449865635599 * jfk_Distance_Pickup  +
0.45000422080251173 * jfk_Distance_Dropoff  +
0.9355834900195628 * sol_Distance_Dropoff  +
-1.1185775883531568 * cp_Distance_Pickup  +
1.553753397052487 * cp_Distance_Dropoff  +
0.8238525341050202 * pickup_near_jfk  +
0.3421228108021715 * dropoff_near_jfk  +
0.8520358752562225 * dropoff_near_ewr  +
1.1556589656135354 * pickup_near_lga  +
0.6477527615076163 * dropoff_near_lga  +
-0.16681821850270015 * pickup_near_esm  +
0.17252397346686255 * dropoff_near_esm  +
-0.20962757928396752 * pickup_near_ts  +
0.13267106888614766 * dropoff_near_ts  

A Cleaner Equation with Coefficients Rounded to 2 decimal Place

In [78]:
# Print the Equation of the model
print("Equation of the model: ")

print("y = ", end="")
for i in range(len(coefficients)):

    print("(", end="")
    print(f"{coefficients[i]:.2f}", end=" ")
    print("*", end=" ")
    print(feature_order[i], end=")")
    print(" ", end="")

    if i < len(coefficients) - 1:
        print("+", end=" ")

Equation of the model: 
y = (11.34 * 1) + (-1.04 * pickup_longitude) + (-0.42 * dropoff_longitude) + (0.97 * year) + (6.85 * H_Distance) + (-1.62 * jfk_Distance_Pickup) + (0.45 * jfk_Distance_Dropoff) + (0.94 * sol_Distance_Dropoff) + (-1.12 * cp_Distance_Pickup) + (1.55 * cp_Distance_Dropoff) + (0.82 * pickup_near_jfk) + (0.34 * dropoff_near_jfk) + (0.85 * dropoff_near_ewr) + (1.16 * pickup_near_lga) + (0.65 * dropoff_near_lga) + (-0.17 * pickup_near_esm) + (0.17 * dropoff_near_esm) + (-0.21 * pickup_near_ts) + (0.13 * dropoff_near_ts) 

**Model-2 : Batch Gradient Descent**

During the Hyper-Parameter Tuning, we have found that the best hyper-parameter is $\eta = 0.1$

In [79]:
batch_gradient_descent = BatchGradientRegressor(learning_rate=0.1)

To Fit Batch Gradient Descent, we need a Validation set.

I will take 10% of the Training Data as Validation Data.

In [80]:
# Import train_test_split
from sklearn.model_selection import train_test_split
X_train_batch, X_val, y_train_batch, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [81]:
batch_gradient_descent.fit(X_train_batch, y_train_batch, X_val, y_val)

Predicting for Test Data

In [82]:
y_pred = batch_gradient_descent.predict(X_test)

Accuracy Score on Test Data

In [84]:
print("RMSE for Test Data: ", np.sqrt(mean_squared_error(y_test, y_pred)))

RMSE for Test Data:  4.792088387469097


Writing the Equation

In [85]:
batch_gradient_descent.theta

array([11.3371823 , -1.05652291, -0.33597433,  0.97033492,  6.81574329,
       -1.65588789,  0.49787812,  0.87921817, -1.14239719,  1.55883173,
        0.83717273,  0.36134181,  0.86691597,  1.16016398,  0.64300068,
       -0.17003514,  0.16860468, -0.21406117,  0.13670618])

Writing the Final Equation:

In [86]:
feature_order = ['1', 'pickup_longitude', 'dropoff_longitude', 'year',
       'H_Distance', 'jfk_Distance_Pickup', 'jfk_Distance_Dropoff',
       'sol_Distance_Dropoff', 'cp_Distance_Pickup', 'cp_Distance_Dropoff',
       'pickup_near_jfk', 'dropoff_near_jfk', 'dropoff_near_ewr',
       'pickup_near_lga', 'dropoff_near_lga', 'pickup_near_esm',
       'dropoff_near_esm', 'pickup_near_ts', 'dropoff_near_ts']
coefficients = list(batch_gradient_descent.theta)

In [87]:
# Print the Equation of the model
print("Equation of the model: ")

print("y = ", end="")
for i in range(len(coefficients)):

    print(coefficients[i], end=" ")
    print("*", end=" ")
    print(feature_order[i], end=" ")
    print(" ", end="")

    if i < len(coefficients) - 1:
        print("+")

Equation of the model: 
y = 11.33718230479152 * 1  +
-1.0565229134175749 * pickup_longitude  +
-0.33597432731010735 * dropoff_longitude  +
0.9703349167862033 * year  +
6.815743285499751 * H_Distance  +
-1.6558878871648717 * jfk_Distance_Pickup  +
0.4978781188180685 * jfk_Distance_Dropoff  +
0.8792181746704661 * sol_Distance_Dropoff  +
-1.1423971914651652 * cp_Distance_Pickup  +
1.5588317333690476 * cp_Distance_Dropoff  +
0.8371727292396821 * pickup_near_jfk  +
0.36134181159893813 * dropoff_near_jfk  +
0.8669159682471761 * dropoff_near_ewr  +
1.1601639787398805 * pickup_near_lga  +
0.643000683784307 * dropoff_near_lga  +
-0.1700351402667336 * pickup_near_esm  +
0.16860468257256525 * dropoff_near_esm  +
-0.21406116853396293 * pickup_near_ts  +
0.13670618328110823 * dropoff_near_ts  

A Cleaner Equation

In [88]:
# Print the Equation of the model
print("Equation of the model: ")

print("y = ", end="")
for i in range(len(coefficients)):

    print("(", end="")
    print(f"{coefficients[i]:.2f}", end=" ")
    print("*", end=" ")
    print(feature_order[i], end=")")
    print(" ", end="")

    if i < len(coefficients) - 1:
        print("+", end=" ")

Equation of the model: 
y = (11.34 * 1) + (-1.06 * pickup_longitude) + (-0.34 * dropoff_longitude) + (0.97 * year) + (6.82 * H_Distance) + (-1.66 * jfk_Distance_Pickup) + (0.50 * jfk_Distance_Dropoff) + (0.88 * sol_Distance_Dropoff) + (-1.14 * cp_Distance_Pickup) + (1.56 * cp_Distance_Dropoff) + (0.84 * pickup_near_jfk) + (0.36 * dropoff_near_jfk) + (0.87 * dropoff_near_ewr) + (1.16 * pickup_near_lga) + (0.64 * dropoff_near_lga) + (-0.17 * pickup_near_esm) + (0.17 * dropoff_near_esm) + (-0.21 * pickup_near_ts) + (0.14 * dropoff_near_ts) 

***Which model is the best for our problem?***

First, all the approaches are giving almost similar results in terms of Accuracy. All the approaches are giving a Mean RMSE of around 4.7 except Scholastic Gradient Descent but that is because of its unstable nature and poor convergence.

Other than that, any approach can outperform any other approach if we tweak with parameters or change the random seeds or increase/decrease data.

Hence, we need to compare all the approaches on some other factor.

If we compare them on the basis of efficiency,

Then definitely optimization based models beat Matrix Based and Non-Parameteric

Since, as we increase the number of rows in our data, Matrix based models will become very slow because of Inverse Operation and Non-Parameteric (KNN) model will require alot of space and will take lots of time in prediction.

Amongst the Optimization based models, clearly Batch Gradient Descent will require lots of memory if we increase data, hence the competition is between Scholastic Gradient Descent and Mini Batch Gradient Descent.

Amongst those two, Mini-Batch Gradient Descent would be better because of faster and more stable convergence.

Also, if we increase the data, then convergence times can be controlled by increasing initial learning rate and controlling the decay properly.